# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0013'

In [3]:
pd.options.display.max_columns = 35

### Create Variables for Queries

In [4]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [7]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null)        
      
    
""")
stdValue_vac = stdVar.first()['std_length_of_stay']
print(stdValue_vac)

5.62


In [ ]:
spark.sql(f"""

          select
                tenant_zip_code
              , count(h.personId) as count_of_non_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( datediff(h.serviceDate, v.vaccine_date) < 0
                    or v.vaccine_date is null)  
               
        group by 1
        order by 1
     
""").toPandas()

In [10]:
spark.sql(f"""

          select
                tenant_zip_code
              , count(h.personId) as count_of_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
               
        group by 1
        order by 1
     
""").toPandas()

,tenant_zip_code,count_of_vaccinated_patients,mean_length_of_stay
0,0,21,4.52
1,1,13,5.54
2,2,44,5.43
3,3,34,4.47
4,4,4,6.00
5,5,30,4.73
6,6,59,5.25
7,7,22,5.77
8,8,82,5.41
9,9,30,6.13


In [11]:
spark.sql(f"""

          select count(h.personId) as count_of_vaccinated_patients
                , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3


              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
               
      
""").toPandas()

,count_of_vaccinated_patients,mean_length_of_stay
0,339,5.28


In [12]:
spark.sql(f"""

          select
               tenant_zip_code
              , count(h.personId) as count_of_all_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}

        group by 1
        order by 1
     
""").toPandas()

,tenant_zip_code,count_of_all_patients,mean_length_of_stay
0,0,9703,7.84
1,1,44900,8.52
2,2,37071,7.93
3,3,51717,7.48
4,4,8400,7.04
5,5,5098,7.26
6,6,15393,7.13
7,7,45894,7.05
8,8,37810,8.07
9,9,37729,7.90


In [13]:
spark.sql(f"""

        select count(h.personId) as count_of_all_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3

     
""").toPandas()

,count_of_all_patients,mean_length_of_stay
0,293715,7.74
